In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

# Constants
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 32
EPOCHS = 30

# Set your dataset paths
COVER_DIR = '/kaggle/input/alaska2-image-steganalysis/Cover'
JMIPOD_DIR = '/kaggle/input/alaska2-image-steganalysis/JMiPOD'
JUNIWARD_DIR = '/kaggle/input/alaska2-image-steganalysis/JUNIWARD'
UERD_DIR = '/kaggle/input/alaska2-image-steganalysis/UERD'

# Load dataset paths and create labels
def load_data_paths():
    image_paths = []
    labels = []
    groups = []

    # Load cover images
    for img_name in os.listdir(COVER_DIR):
        img_path = os.path.join(COVER_DIR, img_name)
        image_paths.append(img_path)
        labels.append(0)  # Cover images
        groups.append(img_name)  # Use image name as group identifier

    # Load stego images
    for img_name in os.listdir(JMIPOD_DIR):
        img_path = os.path.join(JMIPOD_DIR, img_name)
        image_paths.append(img_path)
        labels.append(1)  # JMiPOD images
        groups.append(img_name)  # Group by cover image name

    for img_name in os.listdir(JUNIWARD_DIR):
        img_path = os.path.join(JUNIWARD_DIR, img_name)
        image_paths.append(img_path)
        labels.append(1)  # JUNIWARD images
        groups.append(img_name)  # Group by cover image name

    for img_name in os.listdir(UERD_DIR):
        img_path = os.path.join(UERD_DIR, img_name)
        image_paths.append(img_path)
        labels.append(1)  # UERD images
        groups.append(img_name)  # Group by cover image name

    return np.array(image_paths), np.array(labels), np.array(groups)

# Data Augmentation
def augment(image):
    # Resize image
    image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH))

    # Random flip
    if np.random.rand() > 0.5:
        image = cv2.flip(image, 1)  # horizontal flip

    # Random rotation
    angle = np.random.randint(-10, 11)  # Rotate between -10 and +10 degrees
    center = (IMG_HEIGHT // 2, IMG_WIDTH // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    image = cv2.warpAffine(image, matrix, (IMG_HEIGHT, IMG_WIDTH))

    return image

# Custom Data Generator
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, labels, batch_size, shuffle=True):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.image_paths))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        return self.__data_generation(batch_indices)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, batch_indices):
        batch_images = np.empty((self.batch_size, IMG_HEIGHT, IMG_WIDTH, 3))
        batch_labels = np.empty((self.batch_size,))

        for i, idx in enumerate(batch_indices):
            image = cv2.imread(self.image_paths[idx])
            image = augment(image)  # Apply augmentation
            batch_images[i,] = image / 255.0  # Normalize to [0, 1]
            batch_labels[i] = self.labels[idx]

        return batch_images, batch_labels

# Load Data
image_paths, labels, groups = load_data_paths()

# Group K-Fold Split
gkf = GroupKFold(n_splits=5)
folds = list(gkf.split(image_paths, labels, groups=groups))

# Define the model using a pretrained EfficientNet
def create_model():
    base_model = tf.keras.applications.EfficientNetB0(
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze the base model

    inputs = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # Binary classification
    model = tf.keras.Model(inputs, outputs)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Training Loop
def train_model():
    for fold, (train_idx, val_idx) in enumerate(folds):
        print(f"Training fold {fold + 1}/{len(folds)}")

        # Split data into training and validation
        X_train_paths, X_val_paths = image_paths[train_idx], image_paths[val_idx]
        y_train, y_val = labels[train_idx], labels[val_idx]

        # Create generators
        train_generator = DataGenerator(X_train_paths, y_train, batch_size=BATCH_SIZE)
        val_generator = DataGenerator(X_val_paths, y_val, batch_size=BATCH_SIZE, shuffle=False)

        model = create_model()

        # Define the checkpoint callback to save the entire model after every epoch
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=f'model_fold_{fold + 1}_epoch_{{epoch:02d}}.keras',  # Save as .keras
            save_weights_only=False,  # Save the entire model
            save_best_only=False,  # Save every epoch
            mode='auto',
            verbose=1
        )

        # Training
        model.fit(train_generator, 
                  validation_data=val_generator,
                  epochs=EPOCHS,
                  verbose=1,
                  callbacks=[checkpoint_callback])  # Add the checkpoint callback

        # Evaluate on validation set
        val_preds = model.predict(val_generator)
        val_auc = roc_auc_score(y_val, val_preds)
        print(f"Fold {fold + 1} Validation AUC: {val_auc:.4f}")

if __name__ == "__main__":
    train_model()

Training fold 1/5
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/30


I0000 00:00:1728264662.359964     101 service.cc:145] XLA service 0x7887f8001700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728264662.360016     101 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


   1/7500 ━━━━━━━━━━━━━━━━━━━━ 68:25:21 33s/step - accuracy: 0.7188 - loss: 0.6892

I0000 00:00:1728264680.638097     101 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


  17/7500 ━━━━━━━━━━━━━━━━━━━━ 1:00:13 483ms/step - accuracy: 0.7416 - loss: 0.6481

In [ ]:
import os
import tensorflow as tf

# Define the model loading function
def load_model(fold, epoch):
    """
    Load the model from the specified fold and epoch.
    Arguments:
    - fold: The fold number (1-based index).
    - epoch: The epoch number (1-based index).
    
    Returns:
    - The loaded model.
    """
    model_path = f'model_fold_{fold}_epoch_{epoch:02d}.keras'
    
    if os.path.exists(model_path):
        print(f"Loading model from {model_path}")
        model = tf.keras.models.load_model(model_path)
        return model
    else:
        print(f"No model found at {model_path}. Starting fresh training.")
        return create_model()

# Modified training loop for resuming training
def resume_training(fold, start_epoch=0):
    """
    Resume training for the given fold from the specified epoch.
    Arguments:
    - fold: The fold number to resume training on (1-based index).
    - start_epoch: The epoch to start from (if model is loaded).
    """
    print(f"Resuming training for fold {fold} starting from epoch {start_epoch + 1}")

    # Split data into training and validation
    train_idx, val_idx = folds[fold - 1]
    X_train_paths, X_val_paths = image_paths[train_idx], image_paths[val_idx]
    y_train, y_val = labels[train_idx], labels[val_idx]

    # Create generators
    train_generator = DataGenerator(X_train_paths, y_train, batch_size=BATCH_SIZE)
    val_generator = DataGenerator(X_val_paths, y_val, batch_size=BATCH_SIZE, shuffle=False)

    # Load the model from the latest epoch or create a new model
    model = load_model(fold, start_epoch)

    # Define the checkpoint callback to save the entire model after every epoch
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'model_fold_{fold}_epoch_{{epoch:02d}}.keras',  # Save as .keras
        save_weights_only=False,  # Save the entire model
        save_best_only=False,  # Save every epoch
        mode='auto',
        verbose=1
    )

    # Training
    model.fit(train_generator, 
              validation_data=val_generator,
              initial_epoch=start_epoch,  # Start training from the specified epoch
              epochs=EPOCHS,
              verbose=1,
              callbacks=[checkpoint_callback])  # Add the checkpoint callback

    # Evaluate on validation set
    val_preds = model.predict(val_generator)
    val_auc = roc_auc_score(y_val, val_preds)
    print(f"Fold {fold} Validation AUC: {val_auc:.4f}")

# Example usage to resume training
# Resume training from fold 1, epoch 10
resume_training(fold=1, start_epoch=1)


In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tqdm import tqdm

# Define the test directory and path to your saved model
test_dir = '/kaggle/input/alaska2-image-steganalysis/Test'  # Update with the actual test folder path
model_path = '/kaggle/input/mymodel7/keras/default/1/model_fold_1_epoch_07.keras'  # Path to your trained model

# Load your pre-trained model
model = load_model(model_path)

# Image preprocessing parameters (match these with your training setup)
IMG_SIZE = (256, 256)  # Image size used in training
BATCH_SIZE = 32  # Adjust this based on available memory

# Prepare the list for storing image IDs and their corresponding predictions
submission_data = []

# Get the list of test images
test_images = os.listdir(test_dir)

# Preprocessing and prediction
for img_name in tqdm(test_images, desc="Processing images"):
    img_path = os.path.join(test_dir, img_name)
    
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the same way as in training

    # Make predictions
    prediction = model.predict(img_array)
    
    # Assuming sigmoid activation, prediction gives the probability of stego, take the value
    label = prediction[0][0]
    
    # Append the result to submission_data
    submission_data.append([img_name, label])

# Convert the submission data into a DataFrame
submission_df = pd.DataFrame(submission_data, columns=['Id', 'Label'])

# Save the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

print("Predictions saved to submission.csv")
